In [1]:
import time
import pandas as pd
from gensim import corpora, models, similarities

import word_segmentation


def load_1h_news():
    """
    加载过去一小时内的所有新闻
    """
    # 1小时时间窗口的历史新闻数据分词结果列表[title+content,],时间戳列表[timestamp]
    all_doc_list = []
    all_timestamp_list = []

    # 数据库通过时间范围查询获取一小时的新闻纪录放入all_doc[timestamp+title+content,]
    # 测试代码，使用../data/news0312.csv作为一小时时间窗口的历史新闻数据
    ts = '1526458569'
    
    with open('/data/jupyter/stock/data/news_init.csv', 'r', encoding='utf-8') as data:
        for line in data:
            row_words = line.replace('\n', '').split(',')
            news_ts = ts
            ts = str(int(ts) + 1)
            all_doc_list.append(row_words)
            all_timestamp_list.append(news_ts)
    print("load 1h news success")
    print(all_timestamp_list)

    return all_doc_list, all_timestamp_list


class NewsSimilarity(object):

    def __init__(self):
        self.__all_doc_list, self.__all_timestamp_list = load_1h_news()
        time_start = time.time()
        self.__dictionary = corpora.Dictionary(self.__all_doc_list)
        self.__corpus = [self.__dictionary.doc2bow(doc) for doc in self.__all_doc_list]
        self.__tfidf = models.TfidfModel(self.__corpus)
        time_elapsed = time.time() - time_start
        print('totally cost {:.0f}s'.format(time_elapsed))
        print("tf-idf model has beens build successfully")

    def news_similarity(self, news_seg):
        """
        读取一条新闻的标题和正文，计算该新闻与历史新闻数据的相似度并标记
        :param news_seg: 新闻的标题和正文分词结果（list）
        :return: repeat 是否重复的标记（int）
        """

        # 使用doc2bow将新推送的新闻转换为二元组的向量
        news_vec = self.__dictionary.doc2bow(news_seg)

        # 对corpus语料库中的每个目标文档计算文档的相似度
        index = similarities.SparseMatrixSimilarity(self.__tfidf[self.__corpus], num_features=len(self.__dictionary.keys()))
        sim = index[self.__tfidf[news_vec]]
        print("similarity success")

        # 如果有相似度在90%以上的文档存在，即将新推送的新闻标记为1，否则0
        repeat = 1 if sim.max(axis=0) >= 0.95 else 0

        return repeat

    def add_news(self, news_timestamp, news_seg):
        """
        将新推送的新闻添加至历史数据列表及语料库中
        """
        # 新增推送新闻至all_doc_list（一小时时间窗口历史数据分词结果列表）
        self.__all_timestamp_list.append(news_timestamp)
        self.__all_doc_list.append(news_seg)
        # 新增推送新闻所建立的向量至corpus语料库
        self.__corpus.append(self.__dictionary.doc2bow(news_seg))

    def delete_1h_news(self):
        """
        定时（每小时）删除all_doc_list里前一小时的新闻数据
        """
        time_flag = int(self.__all_timestamp_list[-1]) - 3600
        print(time_flag)

        index = 0
        for ts in self.__all_timestamp_list:
            if int(ts) >= time_flag:
                del_news = index
                break
            else:
                index += 1

        del self.__all_timestamp_list[0:del_news]
        del self.__all_doc_list[0:del_news]
        print(self.__all_timestamp_list)
        
    
    def tf_idf_model(self):
        """
        重新计算tf-idf模型
        """
        self.__dictionary = corpora.Dictionary(self.__all_doc_list)
        self.__corpus = [self.__dictionary.doc2bow(doc) for doc in self.__all_doc_list]
        self.__tfidf = models.TfidfModel(self.__corpus)
        print("tf-idf model has beens build successfully")


if __name__ == "__main__":

    #time_start = time.time()
    # 新推送的新闻
    str_title_content = "孙公司海航地产拟与海南融创昌晟签订《股权转让协议》，出售海航地产所持有的海岛物流100%的股权，转让价款约7.97亿元；同时，海航地产拟出售所持有的海南高和房地产开发有限公司100%的股权至海南融创昌晟，转让价款约11.36亿元。责任编辑：张恒"

    # 新闻分词
    ws = word_segmentation.WordSegmentation()
    news_seg = ws.word_segmentation(str_title_content)
    news_timestamp = '1526523278'
    print("分词结果：")
    print(news_seg)

    # 加载NewsSimilarity类
    news_similarity = NewsSimilarity()

    repeat = news_similarity.news_similarity(news_seg)

    #time_elapsed = time.time() - time_start
    #print('totally cost {:.0f}s'.format(time_elapsed))

    # 将新推送的新闻添加至历史数据列表及语料库中
    news_similarity.add_news(news_timestamp, news_seg)
    print(repeat)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.880 seconds.
Prefix dict has been built succesfully.


分词结果：
['孙', '公司', '海航', '地产', '拟', '海南', '融创昌晟', '签订', '股权', '转让', '协议', '出售', '海航', '地产', '持有', '海岛', '物流', '股权', '转让', '价款', '约', '亿元', '海航', '地产', '拟', '出售', '持有', '海南', '高', '房地', '地产', '房地产', '开发', '有限', '公司', '有限公司', '股权', '海南', '融创昌晟', '转让', '价款', '约', '亿元', '责任', '编辑', '责任编辑', '张恒']
load 1h news success
['1526458569', '1526458570', '1526458571', '1526458572', '1526458573', '1526458574', '1526458575', '1526458576', '1526458577', '1526458578', '1526458579', '1526458580', '1526458581', '1526458582', '1526458583', '1526458584', '1526458585', '1526458586', '1526458587', '1526458588', '1526458589', '1526458590', '1526458591', '1526458592', '1526458593', '1526458594', '1526458595', '1526458596', '1526458597', '1526458598', '1526458599', '1526458600', '1526458601', '1526458602', '1526458603', '1526458604', '1526458605', '1526458606', '1526458607', '1526458608', '1526458609', '1526458610', '1526458611', '1526458612', '1526458613', '1526458614', '1526458615', '1526458616', '1526458617', 

In [2]:
time_start = time.time()
# 新推送的新闻
str_title_content = "孙公司海航地产拟与海南融创昌晟签订《股权转让协议》，出售海航地产所持有的海岛物流100%的股权，转让价款约7.97亿元；同时，海航地产拟出售所持有的海南高和房地产开发有限公司100%的股权至海南融创昌晟，转让价款约11.36亿元。责任编辑：周翎羽"

# 新闻分词
ws = word_segmentation.WordSegmentation()
news_seg = ws.word_segmentation(str_title_content)
news_timestamp = '1526523316'

print("分词结果：")
print(news_seg)

repeat = news_similarity.news_similarity(news_seg)

time_elapsed = time.time() - time_start
print('totally cost {:.0f}s'.format(time_elapsed))

# 将新推送的新闻添加至历史数据列表及语料库中
news_similarity.add_news(news_timestamp, news_seg)
print(repeat)


分词结果：
['孙', '公司', '海航', '地产', '拟', '海南', '融创昌晟', '签订', '股权', '转让', '协议', '出售', '海航', '地产', '持有', '海岛', '物流', '股权', '转让', '价款', '约', '亿元', '海航', '地产', '拟', '出售', '持有', '海南', '高', '房地', '地产', '房地产', '开发', '有限', '公司', '有限公司', '股权', '海南', '融创昌晟', '转让', '价款', '约', '亿元', '责任', '编辑', '责任编辑', '周翎羽']
similarity success
totally cost 1s
1


In [3]:
news_similarity.delete_1h_news()

1526519716
['1526523278', '1526523316']
